In [ ]:
pip install roboflow opencv-python pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 39.2 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [ ]:
!pip install fpdf


  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=fbe5222ed2734695d1f2fa331d1d45a3482c85b3d20d33f8629d5dbaaf16b1dd
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [ ]:
import cv2
import os
from roboflow import Roboflow

# Setup Roboflow
rf = Roboflow(api_key="yGMeddzpCR5eQnoBCBMl")  # Replace with your actual API key
project = rf.workspace().project("cracks-algae-detection")
model = project.version(4).model

# Load video
video_path = "/content/Untitled video - Made with Clipchamp.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Output video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('annotated_output_2.mp4', fourcc, fps, (width, height))

frame_index = 0
frame_gap = 5  # Predict every 5th frame

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame is None:
        print(f"⚠️ Warning: Empty frame at index {frame_index}")
        frame_index += 6
        continue

    frame_path = "temp_frame.jpg"

    if frame_index % frame_gap == 0:
        resized = cv2.resize(frame, (640, 640))
        cv2.imwrite(frame_path, resized)

        # Run model prediction and get annotated image
        prediction = model.predict(frame_path)
        prediction.save("annotated_frame.jpg")

        annotated = cv2.imread("annotated_frame.jpg")
        annotated = cv2.resize(annotated, (width, height))  # Resize back to original size
    else:
        annotated = frame

    out.write(annotated)
    frame_index += 1

cap.release()
out.release()
print("✅ Annotated video saved as: annotated_output.mp4")


loading Roboflow workspace...
loading Roboflow project...
✅ Annotated video saved as: annotated_output.mp4


In [ ]:
import cv2
from PIL import Image
import os

# Input video path
video_path = "annotated_output_2.mp4"  # Update if needed
cap = cv2.VideoCapture(video_path)

output_folder = "frame_screenshots"
os.makedirs(output_folder, exist_ok=True)

frame_gap = 30  # capture every 30th frame
frame_index = 0
image_paths = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_index % frame_gap == 0:
        img_path = os.path.join(output_folder, f"frame_{frame_index}.jpg")
        cv2.imwrite(img_path, frame)
        image_paths.append(img_path)

    frame_index += 1

cap.release()
print(f"✅ {len(image_paths)} frames saved to '{output_folder}' folder.")

# Convert images to PDF
pdf_path = "defect_screenshots_2.pdf"
image_list = [Image.open(img).convert("RGB") for img in image_paths]
if image_list:
    image_list[0].save(pdf_path, save_all=True, append_images=image_list[1:])
    print(f"📄 PDF saved as: {pdf_path}")
else:
    print("⚠️ No images found to include in PDF.")


✅ 0 frames saved to 'frame_screenshots' folder.
⚠️ No images found to include in PDF.
